# Play-out


In [1]:
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator

from pm4py.objects.petri_net.importer import importer as pnml_importer

from pm4py.objects.log.exporter.xes import exporter as xes_exporter

from pm4py.objects.conversion.wf_net import converter as wf_net_converter

from pm4py.algo.filtering.log.end_activities import end_activities_filter


import pandas as pd

import json

from pm4py.algo.simulation.playout.process_tree import algorithm as tree_playout

from pm4py.objects.log.importer.xes import importer as xes_importer


In [2]:
def get_alphabet(net):
    activities = list({a.label for a in net.transitions if a.label and not '_' in a.label})
    return activities

def get_integer_map_net(net):
    return {x: i+1 for i,x in enumerate(get_alphabet(net))}

def log_to_list(log):
    return [[a['concept:name'] for a in t] for t in log]

def apply_integer_map(log, map):
    return [[map[a] for a in t] for t in log]

def save_log(loglist, filename): #save a list of lists 
    df = pd.DataFrame.from_records(loglist)
    df.to_csv(filename, index=False)
    
def add_BOS_EOS(loglist):
    new_log = []
    for i in range(0, len(loglist)):
        new_trace = ['BOS']
        for j in range(0, len(loglist[i])):
            new_trace.append(loglist[i][j])
        new_trace.append('EOS')
        new_log.append(new_trace)
    return new_log

def add_BOS_EOS_to_mapping(mapping):
    all_values = mapping.values()
    max_value = max(all_values)
    mapping["BOS"]= max_value + 1 
    mapping["EOS"]= max_value + 2 
    return mapping
    
    

In [3]:
def read_out_and_play_out(PN_filename, traces_per_variant):
    
    
    net, im, fm = pnml_importer.apply("Nets/"+PN_filename+".pnml")

    variants = simulator.apply(net, im, variant=simulator.Variants.EXTENSIVE, 
                               parameters={simulator.Variants.EXTENSIVE.value.Parameters.MAX_TRACE_LENGTH: 100,
                                          simulator.Variants.EXTENSIVE.value.Parameters.MAX_MARKING_OCC: 3})
    
    xes_exporter.apply(variants, "Variants/XES/Variants_"+PN_filename+".xes")
    
    number_of_variants = len(variants)
    print(number_of_variants)
    
    number_of_traces = number_of_variants * traces_per_variant
    
    simulated_log = simulated_log = simulator.apply(net, im, variant=simulator.Variants.BASIC_PLAYOUT, 
                                                    parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: number_of_traces})
    print(len(simulated_log))
    
    xes_exporter.apply(simulated_log, "Full_Logs/XES/Log_"+PN_filename+".xes")
    
    mapping = get_integer_map_net(net)
    mapping = add_BOS_EOS_to_mapping(mapping)
    #save number mapping
    mappingfilename = 'Mappings/Mapping_'+PN_filename+'.txt'  
    with open(mappingfilename, 'w') as f:
        f.write(json.dumps(mapping))
        
    var_list = apply_integer_map(add_BOS_EOS(log_to_list(variants)), mapping)
    save_log(var_list, "Variants/CSV/Variants_"+PN_filename+".csv")
    log_list = apply_integer_map(add_BOS_EOS(log_to_list(simulated_log)), mapping)
    save_log(log_list, "Full_Logs/CSV/Log_"+PN_filename+".csv")
    
    

In [4]:
def fix_variants_IOR(log):
    end_activities = end_activities_filter.get_end_activities(log)
    filtered_log = end_activities_filter.apply(log, ["O"])
    return(filtered_log)

In [5]:
def read_out_and_play_out_model4(PN_filename, traces_per_variant):
    
    
    net, im, fm = pnml_importer.apply("Nets/"+PN_filename+".pnml")

    variants = simulator.apply(net, im, variant=simulator.Variants.EXTENSIVE, 
                               parameters={simulator.Variants.EXTENSIVE.value.Parameters.MAX_TRACE_LENGTH: 100,
                                          simulator.Variants.EXTENSIVE.value.Parameters.MAX_MARKING_OCC: 3})
    print(len(variants))
    
    variants = fix_variants_IOR(variants)
    
    xes_exporter.apply(variants, "Variants/XES/Variants_"+PN_filename+".xes")
    
    number_of_variants = len(variants)
    print(number_of_variants)
    
    number_of_traces = number_of_variants * traces_per_variant * 10
    
    simulated_log_dum = simulator.apply(net, im, variant=simulator.Variants.BASIC_PLAYOUT, 
                                        parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: number_of_traces})
    
    print(len(simulated_log_dum))
    
    simulated_log_dum = fix_variants_IOR(simulated_log_dum)
    print(len(simulated_log_dum))
    print(type(simulated_log_dum))
    
    xes_exporter.apply(simulated_log_dum, "dum.xes")
    
    number_of_traces = number_of_variants * traces_per_variant
    
    variant = xes_importer.Variants.ITERPARSE
    parameters = {variant.value.Parameters.TIMESTAMP_SORT: True, variant.value.Parameters.MAX_TRACES: number_of_traces}
    simulated_log = xes_importer.apply("dum.xes",variant=variant, parameters=parameters)

    #simulated_log = simulated_log_dum[0:number_of_traces]
    
    print(len(simulated_log))
    print(type(simulated_log))
    
    
    xes_exporter.apply(simulated_log, "Full_Logs/XES/Log_"+PN_filename+".xes")
    
    mapping = get_integer_map_net(net)
    mapping = add_BOS_EOS_to_mapping(mapping)
    #save number mapping
    mappingfilename = 'Mappings/Mapping_'+PN_filename+'.txt'  
    with open(mappingfilename, 'w') as f:
        f.write(json.dumps(mapping))
        
    var_list = apply_integer_map(add_BOS_EOS(log_to_list(variants)), mapping)
    save_log(var_list, "Variants/CSV/Variants_"+PN_filename+".csv")
    log_list = apply_integer_map(add_BOS_EOS(log_to_list(simulated_log)), mapping)
    save_log(log_list, "Full_Logs/CSV/Log_"+PN_filename+".csv")
    

In [6]:
read_out_and_play_out('Model1', 100)

exporting log, completed traces ::   0%|          | 0/120 [00:00<?, ?it/s]

120
12000


exporting log, completed traces ::   0%|          | 0/12000 [00:00<?, ?it/s]

In [7]:
read_out_and_play_out('Model2', 100)

exporting log, completed traces ::   0%|          | 0/128 [00:00<?, ?it/s]

128
12800


exporting log, completed traces ::   0%|          | 0/12800 [00:00<?, ?it/s]

In [8]:
read_out_and_play_out('Model3', 100)

exporting log, completed traces ::   0%|          | 0/128 [00:00<?, ?it/s]

128
12800


exporting log, completed traces ::   0%|          | 0/12800 [00:00<?, ?it/s]

In [9]:
read_out_and_play_out_model4('Model4', 100)

85


exporting log, completed traces ::   0%|          | 0/64 [00:00<?, ?it/s]

64
64000
27043
<class 'pm4py.objects.log.obj.EventLog'>


exporting log, completed traces ::   0%|          | 0/27043 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/27043 [00:00<?, ?it/s]

6400
<class 'pm4py.objects.log.obj.EventLog'>


exporting log, completed traces ::   0%|          | 0/6400 [00:00<?, ?it/s]

In [10]:
read_out_and_play_out('Model5', 100)

exporting log, completed traces ::   0%|          | 0/126 [00:00<?, ?it/s]

126
12600


exporting log, completed traces ::   0%|          | 0/12600 [00:00<?, ?it/s]

In [12]:
read_out_and_play_out('Model6', 100)

exporting log, completed traces ::   0%|          | 0/27 [00:00<?, ?it/s]

27
2700


exporting log, completed traces ::   0%|          | 0/2700 [00:00<?, ?it/s]